In [ ]:
import pymc as pm
import arviz as az
import numpy as np

returns = df['log_return'].dropna().values
n = len(returns)

with pm.Model() as model:
    tau = pm.DiscreteUniform("tau", lower=0, upper=n-1)
    
    mu_1 = pm.Normal("mu_1", mu=0, sigma=1)
    mu_2 = pm.Normal("mu_2", mu=0, sigma=1)
    sigma = pm.HalfNormal("sigma", sigma=1)
    
    mu = pm.math.switch(np.arange(n) < tau, mu_1, mu_2)
    
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=returns)
    
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)


In [ ]:
az.summary(trace)
az.plot_trace(trace)
az.plot_posterior(trace, var_names=["tau"])


In [ ]:
change_date = df['log_return'].dropna().index[int(trace.posterior['tau'].mean())]
print(change_date)


In [ ]:
mu1 = trace.posterior['mu_1'].mean().values
mu2 = trace.posterior['mu_2'].mean().values

impact = (mu2 - mu1) * 100
impact
